In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!/opt/bin/nvidia-smi

Mon Apr 20 16:01:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
googlepath = "/content/gdrive/My Drive/CSYE7374-FinalProject/"

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

!pip install catboost
from catboost import CatBoostClassifier

## Cat Boost (with or without GPU)

In [ ]:
df = pd.read_csv('{}Data/train.csv'.format(googlepath))

In [ ]:
df.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
df = pd.get_dummies(df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['delayed'],axis=1), df.delayed, 
                                                    test_size=0.33, random_state=42, stratify=df.delayed)

In [ ]:
model = CatBoostClassifier(random_state=17, learning_rate=0.1, max_depth=5, verbose=2)

model.fit(X_train, y_train)

model_valid_pred = model.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, model_valid_pred)

0:	learn: 0.6361904	total: 670ms	remaining: 11m 9s
2:	learn: 0.5591067	total: 1.8s	remaining: 9m 58s
4:	learn: 0.5125640	total: 2.87s	remaining: 9m 31s
6:	learn: 0.4831571	total: 3.94s	remaining: 9m 19s
8:	learn: 0.4648672	total: 5.04s	remaining: 9m 15s
10:	learn: 0.4527800	total: 6.19s	remaining: 9m 16s
12:	learn: 0.4450054	total: 7.26s	remaining: 9m 11s
14:	learn: 0.4397218	total: 8.34s	remaining: 9m 7s
16:	learn: 0.4354445	total: 9.4s	remaining: 9m 3s
18:	learn: 0.4321528	total: 10.6s	remaining: 9m 4s
20:	learn: 0.4301873	total: 11.6s	remaining: 9m 1s
22:	learn: 0.4287823	total: 12.7s	remaining: 8m 59s
24:	learn: 0.4277086	total: 13.8s	remaining: 8m 58s
26:	learn: 0.4265736	total: 15s	remaining: 8m 58s
28:	learn: 0.4256553	total: 16s	remaining: 8m 57s
30:	learn: 0.4249990	total: 17.2s	remaining: 8m 56s
32:	learn: 0.4244324	total: 18.2s	remaining: 8m 54s
34:	learn: 0.4238518	total: 19.3s	remaining: 8m 52s
36:	learn: 0.4233422	total: 20.4s	remaining: 8m 50s
38:	learn: 0.4227110	total:

0.7669619882652702

In [ ]:
model_gpu = CatBoostClassifier(random_state=17, learning_rate=0.1, max_depth=5, verbose=2, task_type="GPU")

model_gpu.fit(X_train, y_train)

model_valid_pred_gpu = model_gpu.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, model_valid_pred_gpu)

0:	learn: 0.6360254	total: 33ms	remaining: 33s
2:	learn: 0.5590197	total: 77.9ms	remaining: 25.9s
4:	learn: 0.5112303	total: 123ms	remaining: 24.5s
6:	learn: 0.4821727	total: 167ms	remaining: 23.7s
8:	learn: 0.4649356	total: 211ms	remaining: 23.2s
10:	learn: 0.4516710	total: 255ms	remaining: 22.9s
12:	learn: 0.4440055	total: 300ms	remaining: 22.8s
14:	learn: 0.4385627	total: 344ms	remaining: 22.6s
16:	learn: 0.4348140	total: 388ms	remaining: 22.4s
18:	learn: 0.4325309	total: 432ms	remaining: 22.3s
20:	learn: 0.4301115	total: 476ms	remaining: 22.2s
22:	learn: 0.4286266	total: 520ms	remaining: 22.1s
24:	learn: 0.4273432	total: 564ms	remaining: 22s
26:	learn: 0.4261968	total: 608ms	remaining: 21.9s
28:	learn: 0.4253442	total: 652ms	remaining: 21.8s
30:	learn: 0.4245861	total: 695ms	remaining: 21.7s
32:	learn: 0.4240002	total: 738ms	remaining: 21.6s
34:	learn: 0.4233775	total: 781ms	remaining: 21.5s
36:	learn: 0.4227651	total: 824ms	remaining: 21.5s
38:	learn: 0.4222779	total: 870ms	remain

0.7660014629988159

## LGBM (with GPU)

In [ ]:
#!rm -r /content/LightGBM
!git clone --recursive https://github.com/Microsoft/LightGBM ; cd LightGBM
import os
os.chdir('LightGBM')
!mkdir build
os.chdir('build')
!apt-get install -qq cmake
!cmake -DUSE_GPU=1 ..
!make -j4

Cloning into 'LightGBM'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17436 (delta 5), reused 5 (delta 2), pack-reused 17418
Receiving objects: 100% (17436/17436), 11.84 MiB | 31.82 MiB/s, done.
Resolving deltas: 100% (12697/12697), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.50 MiB | 35.26 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- wo

In [ ]:
%%bash
sudo apt-get -y install python-pip
sudo -H pip install setuptools numpy scipy scikit-learn -U
cd /content/LightGBM/python-package/
sudo python setup.py install --precompile
cd ..

Reading package lists...
Building dependency tree...
Reading state information...
python-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.18.2)
Requirement already up-to-date: scipy in /usr/local/lib/python3.6/dist-packages (1.4.1)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.22.2.post1)
running install
running build
running build_py
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
co

INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
no previously-included directories found matching 'build'
INFO:LightGBM:Installing lib_lightgbm from: ['../lib_lightgbm.so']


In [ ]:
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install --precompile

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-idna python-ipaddress python-keyring
  python-keyrings.alt python-pip-whl python-pkg-resources python-secretstorage
  python-setuptools python-six python-wheel python-xdg
Suggested packages:
  python-crypto-doc python-cryptography-doc python-cryptography-vectors
  python-dbus-dbg python-dbus-doc python-enum34-doc python-gi-cairo
  gnome-keyring libkf5wallet-bin gir1.2-gnomekeyring-1.0 python-fs
  python-gdata python-keyczar python-secretstorage-doc python-setuptools-doc
The following NEW packages will be installed:
  libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-

In [ ]:
!cd LightGBM/python-package/;python3 setup.py install --precompile
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
#!rm -r LightGBM

/bin/bash: line 0: cd: LightGBM/python-package/: No such file or directory
running install
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
running install_lib
INFO:LightGBM:Installing lib_lightgbm from: ['../lib_lightgbm.so']
running install_egg_info
removing '/usr/local/lib/python3.6/dist-packages/lightgbm-2.3.2-py3.6.egg-info' (and everything under it)
Copying lightgbm.egg-info to /usr/local/lib/python3.6/dist-packages/lightgbm-2.3.2-py3.6.egg-info
running in

In [ ]:
df = pd.read_csv('{}Data/train.csv'.format(googlepath))

In [ ]:
df.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
df=df[df.index%2000==0]
df.shape

(2858, 9)

In [ ]:
df = pd.get_dummies(df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['delayed'],axis=1), df.delayed, 
                                                    test_size=0.33, random_state=42, stratify=df.delayed)

In [ ]:
gbm = lgb.LGBMClassifier(n_estimators=200)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1')

[1]	valid_0's l1: 0.292414	valid_0's binary_logloss: 0.463848
[2]	valid_0's l1: 0.290763	valid_0's binary_logloss: 0.459219
[3]	valid_0's l1: 0.289266	valid_0's binary_logloss: 0.455644
[4]	valid_0's l1: 0.288338	valid_0's binary_logloss: 0.453199
[5]	valid_0's l1: 0.287464	valid_0's binary_logloss: 0.451297
[6]	valid_0's l1: 0.286826	valid_0's binary_logloss: 0.450337
[7]	valid_0's l1: 0.285864	valid_0's binary_logloss: 0.449086
[8]	valid_0's l1: 0.285453	valid_0's binary_logloss: 0.449214
[9]	valid_0's l1: 0.285126	valid_0's binary_logloss: 0.449666
[10]	valid_0's l1: 0.285029	valid_0's binary_logloss: 0.449534
[11]	valid_0's l1: 0.28476	valid_0's binary_logloss: 0.449815
[12]	valid_0's l1: 0.284104	valid_0's binary_logloss: 0.449556
[13]	valid_0's l1: 0.283478	valid_0's binary_logloss: 0.449528
[14]	valid_0's l1: 0.283045	valid_0's binary_logloss: 0.45021
[15]	valid_0's l1: 0.282269	valid_0's binary_logloss: 0.448727
[16]	valid_0's l1: 0.281823	valid_0's binary_logloss: 0.448667
[17

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary',
          'nthread': 3,
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error',
          'device': 'gpu',
          'gpu_platform_id': 0,
          'gpu_device_id': 0}

In [ ]:
# Create parameters to search
gridParams = {
    'learning_rate': [0.5, 0.005],
    'n_estimators': [40, 200, 1000],
    'num_leaves': [5, 10, 15],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], 
    'colsample_bytree' : [0.66],
    'subsample' : [0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.4],
    }

In [ ]:
# Create classifier to use
mdl_gpu = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'auc',
          n_jobs = 3, 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'],
          device = params['device'],
          gpu_platform_id = params['gpu_platform_id'],
          gpu_device_id = params['gpu_device_id'])

In [ ]:
grid_gpu = GridSearchCV(mdl_gpu, gridParams,
                    verbose=10,
                    cv=4,
                    n_jobs=-1)
# Run the grid
grid_gpu.fit(df.drop('delayed', axis=1), df.delayed)

Fitting 4 folds for each of 72 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   56.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  7

GridSearchCV(cv=4, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0, device='gpu',
                                      gpu_device_id=0, gpu_platform_id=0,
                                      importance_type='split',
                                      learning_rate=0.1, max_bin=512,
                                      max_depth=-1, min_child_samples=5,
                                      min_child_weight=1, min_split_gain=0.5,
                                      n_estimators=100, n_jobs=3, num_leaves=31,
                                      objective='auc', ran...
             iid='deprecated', n_jobs=-1,
             param_grid={'boosting_type': ['gbdt'], 'colsample_bytree': [0.66],
                         'learning_rate': [0.5, 0.005],
                         'n_estimators': [40, 200, 1000],
                         'num_leaves': [5, 10, 15], 'objective': ['binary'

In [ ]:
grid_gpu.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.66,
 'learning_rate': 0.005,
 'n_estimators': 1000,
 'num_leaves': 15,
 'objective': 'binary',
 'random_state': 501,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'subsample': 0.75}

In [ ]:
gbm_gpu = lgb.LGBMClassifier(**grid_gpu.best_params_, device='gpu',
                                      gpu_device_id=0, gpu_platform_id=0)
gbm_gpu.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)

[1]	valid_0's l1: 0.293991	valid_0's binary_logloss: 0.469654
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 0.293913	valid_0's binary_logloss: 0.469401
[3]	valid_0's l1: 0.293829	valid_0's binary_logloss: 0.469146
[4]	valid_0's l1: 0.29375	valid_0's binary_logloss: 0.468843
[5]	valid_0's l1: 0.293671	valid_0's binary_logloss: 0.468583
[6]	valid_0's l1: 0.293596	valid_0's binary_logloss: 0.468322
[7]	valid_0's l1: 0.29352	valid_0's binary_logloss: 0.468083
[8]	valid_0's l1: 0.293442	valid_0's binary_logloss: 0.467835
[9]	valid_0's l1: 0.293412	valid_0's binary_logloss: 0.467756
[10]	valid_0's l1: 0.293331	valid_0's binary_logloss: 0.467513
[11]	valid_0's l1: 0.293253	valid_0's binary_logloss: 0.467267
[12]	valid_0's l1: 0.293242	valid_0's binary_logloss: 0.467216
[13]	valid_0's l1: 0.29322	valid_0's binary_logloss: 0.467173
[14]	valid_0's l1: 0.293133	valid_0's binary_logloss: 0.466937
[15]	valid_0's l1: 0.293047	valid_0's binary_logloss: 0.466694
[16]	va

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.66,
               device='gpu', gpu_device_id=0, gpu_platform_id=0,
               importance_type='split', learning_rate=0.005, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=15, objective='binary',
               random_state=501, reg_alpha=1, reg_lambda=1, silent=True,
               subsample=0.75, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
gbm_gpu = lgb.LGBMClassifier(**grid_gpu.best_params_, device='gpu')
gbm_gpu.fit(df.drop('delayed', axis=1), df.delayed)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.66,
               device='gpu', importance_type='split', learning_rate=0.005,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=15,
               objective='binary', random_state=501, reg_alpha=1, reg_lambda=1,
               silent=True, subsample=0.75, subsample_for_bin=200000,
               subsample_freq=0)

In [ ]:
roc_auc_score(y_test, gbm_gpu.predict_proba(X_test)[:, 1])

0.8585378889100973